In [2]:
# import libraries 
import requests
from requests import get
import pandas as pd
import urllib.request
import zipfile
import os
from io import StringIO
from apscheduler.schedulers.blocking import BlockingScheduler

In [3]:
def ExtractContentsFromLink():
    url = 'http://data.gdeltproject.org/gdeltv2/lastupdate.txt'
    response = requests.get(url)
    text = response.text
    #print(response.text)
    urlToExtractFile = text.split("\n")[0].split(" ")[2]
    #print(urlToExtractFile)
    filehandle, _ = urllib.request.urlretrieve(urlToExtractFile)
    #print(filehandle)
    zip_file_object = zipfile.ZipFile(filehandle, 'r')
    fileName = zip_file_object.namelist()[0]
    #print(fileName)
    fileContent = zip_file_object.open(fileName).read()
    #print(fileContent)
    Content = str(fileContent,'latin-1')
    readFileDF = pd.read_csv(StringIO(Content), sep='\t', header=None)
    # because parquet must have string column names
    readFileDF.columns = readFileDF.columns.astype(str)
    print(len(readFileDF))
    #readFileDF
    ExtractContentsFromDiskIfExist(readFileDF)

In [4]:
def ExtractContentsFromDiskIfExist(readFileDF):
    if not(os.path.exists('outputOriginal.parquet')):
        print("-----------Inside If-----------")
        #Creating a new parquet file and filling it with data recently extracted
        readFileDF.to_parquet('outputOriginal.parquet')
        #readFileDF.to_parquet('fusionbase.gzip',engine='fastparquet')
        readFileDF.to_parquet('outputMerged.parquet')
        print('created')

    else:
        print("-----------Inside Else-----------------")
        # 
        dfExistingOriginalFile = pd.read_parquet('outputOriginal.parquet')
        print("Length of existing original")
        print(len(dfExistingOriginalFile))
        dfExistingMergedFile = pd.read_parquet('outputMerged.parquet')
        print("Length of existing merged")
        print(len(dfExistingMergedFile))
        # replace existing Original file
        readFileDF.to_parquet('outputOriginal.parquet',compression='gzip')
        # merge with file and put back mergedFile
        # 1. Check with the original's length and merge only then
        #    merge the readFileDf and the dataframe formed by reafFileMerged i.e. outputMerged.parquet
        # 2. remove duplicates
        # 3. write the merged dataframe as parquet at the mentioned location
        # Step 1 : merging dataframes
        mergerdDF = readFileDF.append(dfExistingMergedFile, ignore_index = True)
        print("Length after merging two dataframes")
        print(len(mergerdDF))
        # Step 2. removing duplicates
        duplicateRowsDF = mergerdDF[mergerdDF.duplicated()]
        print(len(duplicateRowsDF))
        mergerdDF.drop_duplicates(inplace=True)
        print("Length after removing duplicates")
        print(len(mergerdDF))
        # Step 3. creating merged file 
        mergerdDF.to_parquet('outputMerged.parquet',compression='gzip')
        print('updated')

In [ ]:
scheduler = BlockingScheduler()
scheduler.add_job(ExtractContentsFromLink, 'interval', minutes=15)
scheduler.start()

In [ ]:
scheduler.shutdown()